In [3]:
import numpy as np
import pandas as pd
import ast 

In [4]:
movies = pd.read_csv('tmdb_5000_movies.csv.zip')
credits = pd.read_csv('tmdb_5000_credits.csv.zip')

FileNotFoundError: [Errno 2] No such file or directory: 'tmdb_5000_movies.csv.zip'

In [ ]:
movies.head()

In [ ]:
credits.head(1)

In [ ]:
movies = movies.merge(credits,on = 'title')

In [ ]:
movies.head(1)

In [ ]:
# genres 
# id
# keywords 
# title 
# overview 
# cast
# crew 

movies = movies[['movie_id','title', 'overview','genres','keywords','cast','crew']]

In [ ]:
movies.info()

In [ ]:
movies.head()

In [ ]:
movies.isnull().sum()

In [ ]:
movies.dropna(inplace = True)

In [ ]:
movies.duplicated().sum()

In [ ]:
movies.iloc[0].genres

In [ ]:
#'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'
#['Action','Adventure','Fantasy','SciFi']

In [ ]:
def convert(obj):
    L = []
    for i in ast.literal_eval(obj):
            L.append(i['name'])
    return L

In [ ]:
movies['genres'].apply(convert)

In [ ]:
movies.head()

In [ ]:

# Safe convert function
def convert(text):
    try:
        return [i['name'] for i in ast.literal_eval(text)]
    except:
        return []

# Fill NaNs first
movies['keywords'] = movies['keywords'].fillna('[]')

# Apply conversion
movies['keywords'] = movies['keywords'].apply(convert)


In [ ]:
def convert(obj):
    L = []
    counter = 0
    for i in ast.literal_eval(obj):
        if counter != 3:  
            L.append(i['name'])
            counter += 1
        else:
            break
    return L

In [ ]:
print(movies['cast'].head())

In [ ]:
movies.head()

In [ ]:
movies['crew'][0]

In [ ]:
def fetch_director(obj):
    L = []
    for i in ast.literal_eval(obj):
        if i['job'] == 'Director':
            L.append(i['name'])
            break
    return L

In [ ]:

def fetch_director(text):
    try:
        for crew_member in ast.literal_eval(text):
            if crew_member['job'] == 'Director':
                return crew_member['name']
    except:
        return ""

# Fix NaNs before applying
movies['crew'] = movies['crew'].fillna('[]')

# Apply safely
movies['crew'] = movies['crew'].apply(fetch_director)


In [ ]:
movies.head()

In [ ]:
movies['overview'][0]

In [ ]:


def safe_extract_genres(x):
    try:
        data = ast.literal_eval(x)
        return [i['name'].replace(" ", "") for i in data if 'name' in i]
    except:
        return []

movies['genres'] = movies['genres'].apply(safe_extract_genres)


In [ ]:
movies.head()

In [ ]:

def clean_column(column):
    def process(x):
        if isinstance(x, str):
            try:
                x = ast.literal_eval(x)
            except:
                return []
        if isinstance(x, list):
            return [i.replace(" ", "") if isinstance(i, str) else "" for i in x]
        return []
    return column.apply(process)

movies['genres'] = clean_column(movies['genres'])
movies['keywords'] = clean_column(movies['keywords'])
movies['cast'] = clean_column(movies['cast'])
movies['crew'] = clean_column(movies['crew'])

In [ ]:
movies.head()

In [ ]:
movies['tags'] = movies['overview'].apply(lambda x: x if isinstance(x, str) else "") + " " + \
                 movies['genres'].apply(lambda x: " ".join(x) if isinstance(x, list) else "") + " " + \
                 movies['keywords'].apply(lambda x: " ".join(x) if isinstance(x, list) else "") + " " + \
                 movies['cast'].apply(lambda x: " ".join(x) if isinstance(x, list) else "") + " " + \
                 movies['crew'].apply(lambda x: " ".join(x) if isinstance(x, list) else "")

In [ ]:
movies.head()

In [ ]:
new_df = movies[['movie_id', 'title', 'tags']]

In [ ]:
new_df['tags'] = new_df['tags'].apply(lambda x: " ".join(x))

In [ ]:
new_df = movies.copy()


In [ ]:
new_df.head()


In [ ]:
new_df['tags'][0]

In [ ]:
new_df['tags'] = new_df['tags'].apply(lambda x:x.lower())

In [ ]:
new_df.head()

In [ ]:
!pip install nltk

In [ ]:
import nltk
from nltk.stem.porter import PorterStemmer 
ps = PorterStemmer()

In [ ]:
 def stem(text):
    y = []
    for i in text.split():
        y.append(ps.stem(i))
    return " ".join(y)


In [ ]:
new_df['tags'] = new_df['tags'].apply(stem)

In [ ]:
new_df['tags'][0]

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(max_features=5000, stop_words='english')

In [ ]:
vectors = cv.fit_transform(new_df['tags']).toarray()


In [ ]:
vectors

In [ ]:
vectors[0]

In [ ]:
cv.get_feature_names_out()


In [ ]:
['loved','loving','love']
['love','love','love']

In [ ]:
ps.stem('danced')

In [ ]:
stem('in the 22nd century, a paraplegic marine is dispatched to the moon pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization. action adventure fantasy sciencefiction cultureclash future spacewar spacecolony society spacetravel futuristic romance space alien tribe alienplanet cgi marine soldier battle loveaffair antiwar powerrelations mindandsoul 3d samworthington zoesaldana sigourneyweaver ')

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
similarity = cosine_similarity(vectors)

In [ ]:
sorted(list(enumerate(similarity[0])),reverse=True,key=lambda x:x[1])[1:6]

In [ ]:
def recommend(movie):
    movie_index = new_df[new_df['title'] == movie].index[0]
    distances = similarity[movie_index]
    movies_list = sorted(list(enumerate(distances)), reverse=True, key=lambda x: x[1])[1:6]
    for i in movies_list:
        print(new_df.iloc[i[0]].title)

In [ ]:
recommend('Avatar')

In [ ]:
new_df.iloc[1216].title 

In [ ]:
import pickle

In [ ]:
pickle.dump(new_df,open('movies.pkl','wb'))

In [ ]:
new_df['title'].values

In [ ]:
pickle.dump(new_df.to_dict(),open('movie_dict.pkl','wb'))

In [ ]:
pickle.dump(similarity, open('similarity.pkl', 'wb'))